In [14]:
import lib.get_assets as assets

In [2]:
import lib.model0_9 as m0_9

### Rule 1

In [3]:

tests = [{'farsi': 'ییلاق‌نشین', 'trans': 'yeylAqneSin'},
         {'farsi': 'ییدیش', 'trans': 'yidiS'},
         {'farsi': 'یوونتوس', 'trans': 'yuventus'},
         {'farsi': 'یونید', 'trans': 'یونید'} # case verb
        ]

for t in tests:   
    assert m0_9.general_search(t['farsi'], pos_neg='Verb') == t['trans']


### Rule 2 (Nouns)

In [4]:

tests = [{'farsi': 'پیامبرت', 'trans': 'payAmbarat'},
         {'farsi': 'مزايایی', 'trans': 'mazAyAyi'},
         {'farsi': 'بی شرف', 'trans': 'bi Saraf'},
         {'farsi': 'بی عقل', 'trans':'bi \'aql'}]

for t in tests:
    assert m0_9.run_transcription_0(t['farsi']) == t['trans']


### Rule 3 (Verbs basis)

In [5]:

tests = [{'farsi': 'بخواهند بروند', 'trans': 'bexAhand beravand'},
         # {'farsi': 'می‌تواند بخوابد', 'trans': 'mitavAnad bexAbad'}, # \u200c
         # {'farsi': 'نشد بپریم', 'trans': 'naSod beparim'}, # می
         # {'farsi': 'می روید', 'trans': 'miravid'} # 
        # {'farsi': 'می ترسد', 'trans': 'mitarsad'}
        ]

for t in tests:
    assert m0_9.run_transcription_0(t['farsi']) == t['trans']


### Rule 4 (Frequency based prioritization for collisions)

In [6]:

tests = [{'farsi': 'ی', 'trans': 'ye'}]

for t in tests:
    assert m0_9.general_search(t['farsi']) == t['trans']

    
tests = [{'farsi': 'ی', 'trans': 'i'}]
for t in tests:
    assert m0_9.affix_search(t['farsi']) == t['trans']


### Rule 5 (Verb _ as خواهند_كر)

In [8]:

tests = [{'farsi': 'خواهند_كرد', 'trans': 'xAhand kard'}]

for t in tests:
    assert m0_9.process_verb(m0_9.normalise(t['farsi'])) == t['trans']

tests = [{'farsi': 'خواهند كرد', 'trans': 'xAhand kard'},
         {'farsi': 'بخواهند بروند', 'trans': 'bexAhand beravand'}]

for t in tests:
    assert m0_9.run_transcription_0(t['farsi']) == t['trans']
    

### Rule 6 (normalisation ي, ك)

In [9]:

tests = [{'farsi': 'گزارشي', 'trans': 'گزارشی'},
         {'farsi': 'ك', 'trans': 'ک'}]

for t in tests:
    assert m0_9.normalise(t['farsi']) == t['trans']


### Rule 7 (affix ی)

### Rule 8 (letter ۀ)

### Rule 9 (collision affix ات)

In [10]:

def rule_9(wrd, pos=None):
    if not 'ات' in wrd:
        return wrd
    else:
        if 'ات‌' in wrd:
            d_affixes = m0_9.get_affixes(wrd, 'ات‌') #, stem)
            stem = 'At'
        elif '\u200cات' in wrd:
            d_affixes = m0_9.get_affixes(wrd, '\u200cات')
            stem = '\'at'
        else:
            d_affixes = m0_9.get_affixes(wrd, 'ات')
            stem = 'At'

    if len(d_affixes['prefix']) > len(d_affixes['suffix']):
        return m0_9.general_search(d_affixes['prefix'], pos_pos=pos) + stem + m0_9.affix_search(d_affixes['suffix'] )
    else:
        return m0_9.affix_search(d_affixes['prefix']) + stem + m0_9.general_search(d_affixes['suffix'], pos_pos=pos)


tests = [{'farsi': 'مزخرفات', 'trans': 'mozaxrafAt'},
         {'farsi': 'خاطرات‌مان', 'trans': 'xAterAtemAn'},
         {'farsi': 'عمه‌ات', 'trans': "'amme'at"}]


for t in tests:
    assert rule_9(m0_9.normalise(t['farsi'])) == t['trans']


### Rule 10 (collision affix ان)

In [52]:

def rule_10(wrd, pos=None):
    if not 'ان' in wrd:
        return wrd
    else:
        if len(wrd) == 2:
            return 'An'
        elif wrd[3] == 'ی':
            return m0_9.general_search(wrd[:-2], pos_pos=pos) + 'yAn'
        else: 
            return m0_9.general_search(wrd[:-2], pos_pos=pos) + 'An'

        
tests = [{'farsi': 'بانیان', 'trans': 'bAniyAn', 'pos': 'Noun'},
         {'farsi': 'دیگران', 'trans': 'digarAn', 'pos': 'Preposition'}]


for t in tests:
    assert rule_10(m0_9.normalise(t['farsi'])) == t['trans']


### Rule 11 (collision affix ش)

In [11]:

def rule_11(wrd, pos=None):
    if 'ش' != wrd[-1]:
        return wrd
    else:
        if pos == 'Noun':
            return m0_9.process_noun(wrd[:-1])+'aS'
        elif pos == 'Verb':
            return m0_9.process_verb(wrd[:-1])+'eS'
        else:
            return m0_9.general_search(wrd[:-1], pos_pos=pos)+'eS'

tests = [{'farsi': 'وسایلش', 'trans':'vasAyelaS', 'pos': 'Noun'},
         {'farsi': 'بردندش', 'trans': 'bordandeS', 'pos': 'Verb'}]

for t in tests:
    assert rule_11(t['farsi'], t['pos']) == t['trans']


### Rule 12 (affix م)

In [65]:


tests = [{'farsi': 'چندم', 'trans':'Candom', 'pos': 'Number'}]

for t in tests:
    assert m0_9.process_wrd(t['farsi'], t['pos']) == t['trans']


### Rule 13 (affix مان)

In [66]:
w = 'مان'

In [69]:
affix = 'مان'
assets.df_Affixes[assets.df_Affixes['Affix']==affix]
#text = 'برای‌مان همه‌ی داستان قهر کردن‌مان و باقی خاطرات‌مان مسخره بود'
#assets.tagger.tag(assets.word_tokenize(text))

,id,Affix,PhonologicalForm,نام وند,کد معنا,جایگاه,نوع,طرح تکیه,واکه,درج واج در وندافزایی,تغییرات آوایی در وندافزایی,SynCatCode,مقولة ستاک+وند,هم‌نویسه با واحد واژگانی,هجاسازی مجدد
99,15,مان,emAn,PEC,03,2,0,W,0,,,N1,,,False
100,129,مان,mAn,,,,1,S,0,,,,,,False


### Rule 14 (affix می)

In [12]:

def rule_14(wrd, pos=None):
    if not 'می' in wrd:
        return wrd
    else:
        if 'می‌' == wrd[:3]:
            return 'mi'+m0_9.process_wrd(wrd[3:], pos)
        elif 'می' == wrd[-2:]:
            return m0_9.general_search(wrd[:-2]) + 'omi'
        
tests = [{'farsi': 'می‌تواند', 'trans': 'mitavAnad', 'pos': 'Verb'},
         {'farsi': 'چندمی', 'trans': 'Candomi'}]

for t in tests:
    assert rule_14(t['farsi'], t.get('pos', None)) == t['trans']


### Rule 15 (affix آ)

### Rule 16 (affix ون)

### Rule 17 (affix ید)

In [15]:

def rule_17(wrd, pos=None):
    if not 'ید' in wrd:
        return wrd
    elif pos == 'Verb':
        l_lemma = [w for w in assets.lemmatizer.lemmatize(wrd).split('#') if w == wrd[:len(w)]]
        if len(l_lemma) > 0:
            # print(1)
            return m0_9.process_verb(wrd[:-len('ید')]) + 'id'
        else:
            return m0_9.process_verb(wrd[:-len('ید')])+'yad'
    else:
        return wrd # ???
    
tests = [{'farsi': 'رفتید', 'trans': 'raftid', 'pos': 'Verb'},
         {'farsi': 'بگوید', 'trans': 'beguyad', 'pos': 'Verb'},
         #{'farsi': 'می‌آید', 'trans': 'beguyad', 'pos': 'Verb'}
        ]

for t in tests:
    assert rule_17(t['farsi'], t.get('pos', None)) == t['trans']


### Rule 18 (verb with یم)

In [16]:

tests = [{'farsi': 'بپریم', 'trans': 'beparim', 'pos': 'Verb'},
         {'farsi': 'رفتیم', 'trans': 'raftim', 'pos': 'Verb'},
         {'farsi': 'خدایم', 'trans': 'xodAyam', 'pos': 'Noun'}]

for t in tests:
    assert m0_9.process_wrd(t['farsi'], t.get('pos', None)) == t['trans']


### Rule 19 (Semispace u200c, improve imlementation in 0.9)

In [97]:
print('چراغ\u200cعلی')

چراغ‌علی


In [ ]:
'چراغ‌علی'
'CerAq\'ali'

In [77]:
#def handle_u200c_exception(wrd):
wrd = 'چراغ\u200cعلی'
stem = '\u200c'
d_affixes = m0_9.get_affixes(wrd, stem)
d_affixes
"""
    if sum([d_affixes['suffix'][-len(s):]==s for s in ['ها']]) > 0:
        #return flow_nouns(d_affixes['prefix']) + affix_choice_model(d_affixes['suffix'])
        return flow_nouns(d_affixes['prefix']) + affix_choice_model(d_affixes['suffix'], 'Noun')
    
    if sum([d_affixes['prefix'][-len(s):]==s for s in ['می', 'نمی']]) > 0:
        # print(1)
        #return affix_choice_model(d_affixes['prefix']) + flow_verb(d_affixes['suffix']) 
        return affix_choice_model(d_affixes['prefix'], 'Verb') + flow_verb(d_affixes['suffix'])    
    
    # print('EXCEPT!!!!!!!!', wrd)
    # print(1 if len(d_affixes['prefix']) >  len(d_affixes['suffix']) else 2)
    #return flow_nouns(d_affixes['prefix']) + affix_choice_model(d_affixes['suffix']) \
    #    if len(d_affixes['prefix']) >  len(d_affixes['suffix']) else \
    #    affix_choice_model(d_affixes['prefix'])+flow_nouns(d_affixes['suffix'])
    return flow_nouns(d_affixes['prefix']) + general_search(d_affixes['suffix']) \
        if len(d_affixes['prefix']) >  len(d_affixes['suffix']) else \
        general_search(d_affixes['prefix'])+flow_nouns(d_affixes['suffix'])
""";


In [124]:
wrd = 'بی\u200cبصیرتی\u200cهایی'
'bibasiratihAyi'

In [126]:
l_wrd = wrd.split('\u200c')
l_wrd

['بی', 'بصیرتی', 'هایی']

In [145]:
m0_9.general_search('بی') + m0_9.process_noun('بصیرتی')

'bibasirati'

In [147]:
#assets.df_Entries[assets.df_Entries['WrittenForm']=='یی'], assets.df_Entries[assets.df_Entries['WrittenForm']=='ها']

In [148]:
assets.stemmer.stem('هایی')

''

In [143]:

assets.df_Affixes[assets.df_Affixes['Affix']=='یی'], assets.df_Affixes[assets.df_Affixes['Affix']=='ها']


(     id Affix PhonologicalForm نام وند کد معنا جایگاه نوع طرح تکیه واکه  \
 163  10    یی               yi     INM      01      2   0        W    1   
 164  35    یی               yi     COC      05      3   0        W    1   
 
     درج واج در وندافزایی تغییرات آوایی در وندافزایی SynCatCode مقولة ستاک+وند  \
 163                                                         N1                  
 164                                                         N1                  
 
     هم‌نویسه با واحد واژگانی هجاسازی مجدد  
 163                                 False  
 164                                 False  ,
       id Affix PhonologicalForm نام وند کد معنا جایگاه نوع طرح تکیه واکه  \
 120    2    ها               hA     PLM      00      1   0        S    1   
 121  171    ها               hA     PLM      00      1   0        S    0   
 
     درج واج در وندافزایی تغییرات آوایی در وندافزایی SynCatCode مقولة ستاک+وند  \
 120                                                         N1        

In [76]:
text = 'امروز بی‌بصیرتی‌هایی دیده می‌شود که چراغ‌علی هم قبلاً ندیده!'
text = m0_9.normalise(text)
text
assets.tagger.tag(assets.word_tokenize(text))

[('امروز', 'ADV'),
 ('بی\u200cبصیرتی\u200cهایی', 'N'),
 ('دیده_می\u200cشود', 'V'),
 ('که', 'CONJ'),
 ('چراغ\u200cعلی', 'N'),
 ('هم', 'CONJ'),
 ('قبلا', 'ADV'),
 ('ندیده', 'V'),
 ('!', 'PUNC')]

### Rule 20 (Recursive search, improve implemented in 0.9)

In [172]:

def recu_affix_wrd(wrd):
    
    suffix, stem = '', ''
    for i in range(len(wrd), 0, -1):
        if m0_9.assets.df_Affixes[m0_9.assets.df_Affixes['Affix']==wrd[:i]].shape[0] > 0:
            l_search = m0_9.assets.df_Affixes[m0_9.assets.df_Affixes['Affix']==wrd[:i]].to_dict('records')
            stem = m0_9.votation_entries(l_search, entries=False)
            suffix = recu_affix_wrd(wrd[i:])
            break

    return stem+suffix

tests = [{'farsi': 'هایی', 'trans': 'hAyi'},
         {'farsi': 'هایتان', 'trans': 'hAyetAn'}]

for t in tests:
    assert recu_affix_wrd(t['farsi']) == t['trans']


### Rule 21 (affix ن)

In [194]:

def rule_21(wrd, pos=None):
    if not (wrd[-1] == 'ن' or wrd[0] == 'ن'):
        return m0_9.process_wrd(wrd, pos)
    else:
        if wrd[-1] == 'ن':
            return rule_21(wrd[:-1], pos='Verb') + 'an'
        elif wrd[0] == 'ن':
            return 'na' + rule_21(wrd[1:], pos='Verb')


tests = [{'farsi': 'نخوردن', 'trans': 'naxordan'},
         {'farsi': 'نخوابیدن', 'trans': 'naxAbidan'},
         #{'farsi': 'نمان', 'trans': 'namAn'}
         ]

for t in tests:
    assert rule_21(t['farsi']) == t['trans']


### Rule 22 (affixes بی and نی)

In [207]:
affix = 'بی'
affix = 'نی'
m0_9.assets.df_Affixes[m0_9.assets.df_Affixes['Affix']==affix]

,id,Affix,PhonologicalForm,نام وند,کد معنا,جایگاه,نوع,طرح تکیه,واکه,درج واج در وندافزایی,تغییرات آوایی در وندافزایی,SynCatCode,مقولة ستاک+وند,هم‌نویسه با واحد واژگانی,هجاسازی مجدد
117,60,نی,nay,NEM,10,-2,0,S,0,,,020,,,False


In [198]:
text = 'بیا در آغوشم بیارام و دیگران را نیازار'

In [208]:
for w in assets.tagger.tag(assets.word_tokenize(text)):
    #print('نی' in w[0])
    print(w)

('بیا', 'V')
('در', 'P')
('آغوشم', 'N')
('بیارام', 'V')
('و', 'CONJ')
('دیگران', 'PRO')
('را', 'POSTP')
('نیازار', 'N')


In [221]:
wrd = 'نیازار'
wrd[:2] == 'نی'
wrd[:-3]

'نیا'

In [220]:
wrd = 'نیازار'
lemma = assets.lemmatizer.lemmatize(wrd[:-3])
lemma

'آمد#آ'

In [215]:

assets.df_Entries[assets.df_Entries['WrittenForm']==wrd[2:]]

,PhonologicalForm,WrittenForm,SynCatCode,Freq,الگوی تکیه
3550,'ezAr,ازار,N1,1,WS


In [213]:
m0_9.process_verb(wrd[2:])

"'ezAr"

In [212]:
m0_9.process_noun(wrd[2:])

"'ezAr"

In [206]:
'nayAzAr'

'nayAzAr'

### Rule 23 (root of the verb is رو)

In [222]:
w='رو'
def rule_23(wrd, pos=None):
    if not wrd[-2:] == 'رو':
        

In [223]:
assets.df_Entries[assets.df_Entries['WrittenForm']==w]

,PhonologicalForm,WrittenForm,SynCatCode,Freq,الگوی تکیه
26085,rav,رو,V1,5000,S
26086,ru,رو,N1,7050,S
26087,ru,رو,N9,1,S
26088,ru,رو,V1,4000,S


In [224]:
text = 'برو بمیر! نرو نمیر!'

In [225]:
assets.tagger.tag(assets.word_tokenize(text))

[('برو', 'V'),
 ('بمیر', 'N'),
 ('!', 'PUNC'),
 ('نرو', 'V'),
 ('نمیر', 'N'),
 ('!', 'PUNC')]

In [227]:
wrd = 'برو'
m0_9.process_verb(wrd)

'boro'

In [228]:
assets.stemmer.stem(wrd)

'برو'

In [235]:
lemma = assets.lemmatizer.lemmatize(wrd)
lemma

'برو'

In [232]:
assets.df_Entries[assets.df_Entries['WrittenForm']==affix]

,PhonologicalForm,WrittenForm,SynCatCode,Freq,الگوی تکیه
26085,rav,رو,V1,5000,S
26086,ru,رو,N1,7050,S
26087,ru,رو,N9,1,S
26088,ru,رو,V1,4000,S


In [231]:
affix = 'ب'
affix = 'رو'
assets.df_Affixes[assets.df_Affixes['Affix']==affix]

,id,Affix,PhonologicalForm,نام وند,کد معنا,جایگاه,نوع,طرح تکیه,واکه,درج واج در وندافزایی,تغییرات آوایی در وندافزایی,SynCatCode,مقولة ستاک+وند,هم‌نویسه با واحد واژگانی,هجاسازی مجدد


In [ ]:
'نرو'

In [236]:
affix = 'ن'
assets.df_Entries[assets.df_Entries['WrittenForm']==affix]

,PhonologicalForm,WrittenForm,SynCatCode,Freq,الگوی تکیه
48877,n,ن,T,1,None
48878,ne,ن,Ab,1,S
48879,nun,ن,Al,1,S


In [237]:
assets.df_Affixes[assets.df_Affixes['Affix']==affix]

,id,Affix,PhonologicalForm,نام وند,کد معنا,جایگاه,نوع,طرح تکیه,واکه,درج واج در وندافزایی,تغییرات آوایی در وندافزایی,SynCatCode,مقولة ستاک+وند,هم‌نویسه با واحد واژگانی,هجاسازی مجدد
105,59,ن,na,NEM,10,-2,0,S,0,,,020,,,False
106,166,ن,na,,,,1,W,0,,,,,,False
107,None,ن,an,,,,None,None,None,,,V2,,,False


In [238]:

assets.df_Affixes[assets.df_Affixes['Affix']=='ب']

,id,Affix,PhonologicalForm,نام وند,کد معنا,جایگاه,نوع,طرح تکیه,واکه,درج واج در وندافزایی,تغییرات آوایی در وندافزایی,SynCatCode,مقولة ستاک+وند,هم‌نویسه با واحد واژگانی,هجاسازی مجدد
33,57,ب,be,ISM,11,-1,0,S,0,,,,,,False


### Rule 24 (Recursion, improve v0.9)